# load Dataset

In [7]:
import numpy as np 
import pickle 

# load dataset 
# with open('../sample_image/sample560_crop_df.bin', 'rb') as f :
#     crop_img = pickle.load(f)

with open('../Data/final_df.bin', 'rb') as f :
    crop_img = pickle.load(f)

    
# X = np.array([i for i in crop_img.crop_img_03])
X = np.array([i for i in crop_img.crop_images])
y = crop_img.weight.values

idx = list(np.random.choice(range(len(X)), int(len(X) * 0.8), replace=False))
test_idx = list(set(list(range(len(X)))) - set(idx))

X_train = X[idx]
y_train = y[idx]

X_test = X[test_idx]
y_test = y[test_idx]

In [8]:
X_train.shape

(448, 250, 550, 3)

# Load Model 

In [12]:
from keras.models import load_model

KERAS_MODEL_FILE = '../model_hdf5/DenseNet_0725_51Epochs_logcosh.hdf5'

model = load_model(KERAS_MODEL_FILE)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
model.count_params()

40681

# Prediction

In [8]:
%%timeit
pred = model.predict(X_test)

5.88 s ± 163 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
pred = model.predict(X_test)
print("MAE :", np.mean(np.abs((y_test.reshape(-1,1) - pred))))

MAE : 2.49695295878819


In [28]:
# model.layers

### 일부가 손상된 사진의 출력 값 확인

In [38]:
import cv2 

sample_collapsed = cv2.imread("/Users/MAC/Desktop/study/영상분류-영상인식/강의자료/23_114_half.png")

sample_collapsed = cv2.resize(sample_collapsed, (550, 250))

# plt.imshow(sample_collapsed[:,:,::-1])

In [40]:
sample_collapsed[np.newaxis,:,:,:].shape

(1, 250, 550, 3)

In [41]:
model.predict(sample_collapsed[np.newaxis,:,:,:]) # 실제 114

array([[106.37561]], dtype=float32)

### 케라스로 만든 모델을 TFlite 로 convert 했을 때

In [33]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model_file(KERAS_MODEL_FILE)
converter.target_ops = [tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
#                         tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

INFO:tensorflow:Froze 104 variables.
INFO:tensorflow:Converted 104 variables to const ops.


In [34]:
with open('../model_hdf5/DenseNet_0720_150Epochs.tflite', 'wb') as f:
    f.write(tflite_model)

In [36]:
interpreter = tf.contrib.lite.Interpreter(model_path="../model_hdf5/DenseNet_0720_150Epochs.tflite")
interpreter.allocate_tensors()

RuntimeError: Regular TensorFlow ops are not supported by this interpreter. Make sure you invoke the Flex delegate before inference.Node number 0 (Flex) failed to prepare.


In [ ]:
# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
correct = 0

for img, label in tqdm(zip(x_test, y_test)):
    interpreter.set_tensor(input_details[0]['index'], img)

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if np.argmax(output_data) == np.argmax(label):
        correct += 1

print('Accuracy:', correct / len(y_test))